<a href="https://colab.research.google.com/github/FabioC-GH/Web-Scrapper-project/blob/master/web_scrapper_cidades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import requests
from bs4 import BeautifulSoup

from openpyxl import Workbook
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#obtains list of cities from Portugal

# requets the page
page_cidades_rec = requests.get("https://www.numbeo.com/cost-of-living/country_result.jsp?country=Portugal")

# parces the html
soup = BeautifulSoup(page_cidades_rec.content, 'html.parser')

# extracts text from html based on tag and id
cidade_rec_var = [x.text for x in soup.find_all('select', {'id': 'city'})]

# splits the list base on '\n' separaint city names from each other
lista_cidade_temp = [i.split('\n') for i in cidade_rec_var]

# deletes non city values
del lista_cidade_temp[0][0:2]
del lista_cidade_temp[0][-1]
lista_cidades_base = lista_cidade_temp[0]

# There are various types of url, requiring to create lists to accommodate those differences
#this will require further data cleaning in the output file (duplicates, empty cells)

#replaces ' ' por '-'
lista_cidades = []
for string in lista_cidades_base:
  new_string = string.replace(" ", "-") #Modify old string.
  lista_cidades.append(new_string) #Add new string to list.

#replaces 'de' por 'De'
lista_cidades_De = []
for string in lista_cidades:
  new_string = string.replace("de", "De") #Modify old string.
  lista_cidades_De.append(new_string) #Add new string to list.

#replaces "do" por "Do"
lista_cidades_Do = []
for string in lista_cidades:
  new_string = string.replace("do", "Do") #Modify old string.
  lista_cidades_Do.append(new_string) #Add new string to list.


In [29]:
#
# This function (url_storage()) is used to store url and there corresponding list
# not meant to be used

def url_storage():
  #url type: https://www.numbeo.com/cost-of-living/in/Braga
  page_cidades = requests.get("https://www.numbeo.com/cost-of-living/in/%s?displayCurrency=EUR" % lista_cidades_base[i])

  #url type: https://www.numbeo.com/cost-of-living/in/Braga com "-" em vez de " "
  page_cidades = requests.get("https://www.numbeo.com/cost-of-living/in/%s&displayCurrency=EUR" % lista_cidades[i])

  #url type: https://www.numbeo.com/cost-of-living/in/Albufeira-Portugal
  page_cidades = requests.get("https://www.numbeo.com/cost-of-living/in/%s-Portugal?displayCurrency=EUR" % lista_cidades[i])

  #url type: contains De
  page_cidades = requests.get("https://www.numbeo.com/cost-of-living/in/%s-Portugal?displayCurrency=EUR" % lista_cidades_De[i])

  #url type: contains Do
  page_cidades = requests.get("https://www.numbeo.com/cost-of-living/in/%s-Portugal?displayCurrency=EUR" % lista_cidades_Do[i])

  #https://www.numbeo.com/cost-of-living/in/Viana-do-Castelo?displayCurrency=EUR
  # url for viana do castelo(did not show up in any of the other results)
  page_cidades = requests.get("https://www.numbeo.com/cost-of-living/in/Viana-do-Castelo?displayCurrency=EUR")

# tuple to be use by the extaction function, extract_values_cities(request_url,chosen_list_cities)
# contains url and corresponding city list
url_tuple =(("https://www.numbeo.com/cost-of-living/in/%s?displayCurrency=EUR", lista_cidades_base),
            ("https://www.numbeo.com/cost-of-living/in/%s&displayCurrency=EUR", lista_cidades),
            ("https://www.numbeo.com/cost-of-living/in/%s-Portugal?displayCurrency=EUR", lista_cidades),
            ("https://www.numbeo.com/cost-of-living/in/%s-Portugal?displayCurrency=EUR", lista_cidades_De),
            ("https://www.numbeo.com/cost-of-living/in/%s-Portugal?displayCurrency=EUR", lista_cidades_Do)
            )

In [30]:
# data extration fuction
# requires 2 parameters, url and list

def extract_values_cities(request_url,chosen_list_cities):

  for i in range(len(chosen_list_cities)):
    page_cidades = requests.get(request_url % chosen_list_cities[i])
      
    page_cidades

    soup = BeautifulSoup(page_cidades.content, 'html.parser')

    # finds all <td> with "priceValues" class, contain prices
    z = soup.find_all('td', class_="priceValue")
    list(z)

    # list to outputed to excel file
    lista_val = []

    print(chosen_list_cities[i])

    # appends city name to output list
    lista_val.append(chosen_list_cities[i])

    # appends text extracted to output list
    for x in range(len(z)):      
      lista_val.append(z[x].get_text())
      
    ## excel append section ##

    #path to the excel file
    excel_file_c1 = '/content/drive/My Drive/Colab Notebooks/webScrapper proj/Cidades_WS_teste.xlsx'

    #load excel file
    wb_c1 = load_workbook(excel_file_c1)

    #declares the active sheet were to append
    f1 = wb_c1.active
    #appends values to declared sheet
    f1.append(lista_val)

    #saves changes made to the excel file
    wb_c1.save(filename=excel_file_c1)

In [ ]:
#function call

#Iterates to a tuple containing URLs and respective list, calling the extraction function
for i in range(len(url_tuple)):
  extract_values_cities(url_tuple[i][0],url_tuple[i][1])